In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-10 14:32:44.952399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 14:32:45.054068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 14:32:45.054086: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 14:32:45.074476: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 14:32:45.597507: W tensorflow/stream_executor/platform/de

In [2]:
name_all=[]
auc_roc=[]
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)

read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
y_m=a1
X_m=main_all_feature

read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
y_e=a2
X_e=external_all_feature

model=SVC(C=1,gamma=0.05,class_weight='balanced',probability=True)

model.fit(X_m,y_m)

proba = model.predict_proba(X_e)
pro=[]
for p in proba:
    pi = p[1]
    pro.append(pi)
y_label = y_e
y_pred = pro
y_label = np.array(y_label)
y_pred = np.array(y_pred)    
fpr = dict()
tpr = dict() 
roc_auc = dict()
fpr, tpr, thresholds = roc_curve(y_label, y_pred)
score=auc(fpr,tpr)
print(score)
model_name='svm'
name_all.append(model_name)
auc_roc.append(score)
model=''

0.8336426687506104


In [3]:
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)

read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
y_m=a1
X_m=main_all_feature

read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
y_e=a2
X_e=external_all_feature

xgboost_model = xgboost.XGBClassifier(
  max_depth=5,
  learning_rate=0.05,
  n_estimators=1000,
  gamma=0.0,
  min_child_weight=5,
  max_delta_step=1,
  subsample=0.53,
  colsample_bytree=0.66,
  colsample_bylevel=1,
  reg_alpha=0,
  reg_lambda=1,
  scale_pos_weight=1,
  base_score=0.5,
  seed=2016)

xgboost_model.fit(X_m, y_m)

proba = xgboost_model.predict_proba(X_e)
pro=[]
for p in proba:
    pi = p[1]
    pro.append(pi)
y_label = y_e
y_pred = pro
y_label = np.array(y_label)
y_pred = np.array(y_pred)    
fpr = dict()
tpr = dict() 
roc_auc = dict()
fpr, tpr, thresholds = roc_curve(y_label, y_pred)
score=auc(fpr,tpr)
print(score)
model_name='xgb'
name_all.append(model_name)
auc_roc.append(score)
model=''


0.8217251147797207


In [4]:
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)

read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
y_m=a1
X_m=main_all_feature

read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
y_e=a2
X_e=external_all_feature

model=RandomForestClassifier(bootstrap=True, class_weight='balanced',
        criterion='gini', max_depth=None, max_features='auto',
        max_leaf_nodes=None, min_impurity_decrease=0.0,
        min_samples_leaf=1,min_samples_split=2,
        min_weight_fraction_leaf=0.0,
        n_estimators=50, n_jobs=-1, oob_score=False, random_state=None,
        verbose=0, warm_start=False)

model.fit(X_m,y_m)

proba = model.predict_proba(X_e)
pro=[]
for p in proba:
    pi = p[1]
    pro.append(pi)
y_label = y_e
y_pred = pro
y_label = np.array(y_label)
y_pred = np.array(y_pred)    
fpr = dict()
tpr = dict() 
roc_auc = dict()
fpr, tpr, thresholds = roc_curve(y_label, y_pred)
score=auc(fpr,tpr)
print(score)
model_name='rf'
name_all.append(model_name)
auc_roc.append(score)
model=''

0.7947966526651689


/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [5]:
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)

read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
y_m=a1
X_m=main_all_feature

read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
y_e=a2
X_e=external_all_feature

sklearn_model = LogisticRegression(
      penalty= 'l2',
      C=1.0,
      class_weight="balanced",
      dual=False,
      fit_intercept=True,
      max_iter=100,
      multi_class='ovr',
      n_jobs=-1,
      random_state=None,solver='liblinear',tol=0.0001,verbose=0,warm_start=False)

sklearn_model.fit(X_m, y_m)

proba = sklearn_model.predict_proba(X_e)
pro=[]
for p in proba:
    pi = p[1]
    pro.append(pi)

y_label = y_e
y_pred = pro
y_label = np.array(y_label)
y_pred = np.array(y_pred)    
fpr = dict()
tpr = dict() 
roc_auc = dict()
fpr, tpr, thresholds = roc_curve(y_label, y_pred)
score=auc(fpr,tpr)
print(score)
model_name='lr'
name_all.append(model_name)
auc_roc.append(score)
model=''

0.8841131842011006


/home/hasee2/miniconda3/envs/deepchem/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


In [6]:
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
model_name='fcnn'
name_all.append(model_name)
auc_roc.append(score[1])
model=''

Epoch 1/100


2022-11-10 14:32:54.220318: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 14:32:54.220930: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-10 14:32:54.220985: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-10 14:32:54.221033: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-10 14:32:54.221078: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 393ms/step - loss: 6.1118 - auc: 0.2045
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 3.2248 - auc: 0.4527
Epoch 3/100
1/1 [==============================] - 0s 35ms/step - loss: 2.2933 - auc: 0.9170
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 2.4064 - auc: 0.9312
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 2.6038 - auc: 0.9184
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 2.7042 - auc: 0.9051
Epoch 7/100
1/1 [==============================] - 0s 36ms/step - loss: 2.7255 - auc: 0.9001
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 2.7461 - auc: 0.8940
Epoch 9/100
1/1 [==============================] - 0s 37ms/step - loss: 2.6690 - auc: 0.8995
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 2.5502 - auc: 0.9061
Epoch 11/100
1/1 [==============================] - 0s 36ms/step - loss: 2.3985 